# Testing Notebook 

This notebook is only for testing porpouse of new code. Training the model should be done with the Python script. 

## Usefull Links

- Finetuning Bert with Sem Eval Task von 2018: https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb

## Imports

In [11]:
%pip install tensorboardX

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)
Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install transformers==3.0.2

  Using cached transformers-3.0.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.8.1rc1.tar.gz (97 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sentencepiece-0.2.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
Using cached sentencepiece-0.2.0-cp39-cp39-macosx_11_0_arm64.whl (1.2 MB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      /private/var/folders/sv/z75_tjbd6dqbww2lwprqfs8h0000gn/T/pip-build-env-hmq0sjfk/overlay/lib/python3.9/site-packages/setuptools/

In [2]:
%pip install 'accelerate>=0.26.0'

Note: you may need to restart the kernel to use updated packages.


In [19]:
from torch import nn
from torch import cuda

import pandas as pd

from sklearn.model_selection import train_test_split
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer, TrainingArguments, Trainer
import logging
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
logging.basicConfig(level=logging.ERROR)


# Setting up the device for GPU usage
device = 'cuda' if cuda.is_available() else 'cpu'

## Prepare Dataset
### For Custom Trainings Loop

First some key variables are definded which will are used later in the custom training loop. 

In [4]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
# EPOCHS = 1
LEARNING_RATE = 1e-05

Create custom dataset class.

In [5]:
class SentimentDataSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer:RobertaTokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.anger= self.data.Anger
        self.fear= self.data.Fear
        self.joy= self.data.Joy
        self.sadness= self.data.Sadness
        self.surprise= self.data.Surprise
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        targets = torch.tensor([self.anger[index], self.fear[index], self.joy[index], self.sadness[index], self.surprise[index]], dtype=torch.float)


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'anger': torch.tensor(self.anger[index], dtype=torch.float),
            'fear': torch.tensor(self.fear[index], dtype=torch.float),
            'joy': torch.tensor(self.joy[index], dtype=torch.float),
            'sadness': torch.tensor(self.sadness[index], dtype=torch.float),
            'surprise': torch.tensor(self.surprise[index], dtype=torch.float),
            'targets': targets,
            'text': text
        }

The data is loaded from the csv file and split into a training and validation set. With the data the custom dataset class is initialized and dataloaders are created. Those dataloaders are later used in the custom traingsloop. 

In [ ]:
train = pd.read_csv('data/public_data/train/track_a/eng.csv')
print(train)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

train = train.dropna()
train = train.drop(0)
train = train.reset_index(drop=True)

training_set, validation_set = train_test_split(train, test_size=0.2, random_state=123)
training_set = training_set.reset_index(drop=True)
validation_set = validation_set.reset_index(drop=True)

training_set_set = SentimentDataSet(training_set, tokenizer, MAX_LEN)
validation_set_set = SentimentDataSet(validation_set, tokenizer, MAX_LEN)

train_dataloader = DataLoader(training_set_set, batch_size=TRAIN_BATCH_SIZE, num_workers=0)
val_dataloader = DataLoader(validation_set_set, batch_size=VALID_BATCH_SIZE, num_workers=0)

# validate the dataset
# print(f"{training_set_set[12]['text']}: {training_set_set[12]['targets']}")

<class 'pandas.core.frame.DataFrame'>


### For Hugginfaces Trainer Class

In [7]:

# All values for each emotion class are combined to a tensor 
# so that the first entry in the tensor is the label of the first 
# text entry in the corresponding test or validation set. 
train_targets = torch.tensor(
    list(zip(
        training_set["Anger"],
        training_set["Fear"],
        training_set["Joy"],
        training_set["Sadness"],
        training_set["Surprise"]
    )),
    dtype=torch.float
)

val_targets = torch.tensor(
    list(zip(
        validation_set["Anger"],
        validation_set["Fear"],
        validation_set["Joy"],
        validation_set["Sadness"],
        validation_set["Surprise"]
    )),
    dtype=torch.float
)

# The test and validation sets are tokenized and the corresponding emotion label is added 
# to every sentence. 
def prepare_data(text, label):
    tokenized = tokenizer(text, truncation=True, add_special_tokens=True,padding='max_length', max_length=MAX_LEN,return_token_type_ids=True, return_tensors="pt")
    tokenized['labels'] = label
    return tokenized

tokenized_train = [
    prepare_data(text, label) for text, label in zip(training_set["text"], train_targets)
]

tokenized_val = [
    prepare_data(text, label) for text, label in zip(validation_set["text"], val_targets)
]


data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
#collated_train = data_collator(tokenized_train)

## Initialize LLM Models

### RoBerta Base Model With Custom Attention Head

In [8]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        classifier = self.classifier(pooler)
        output = self.sigmoid(classifier)
        return output

Test the model.

In [9]:
model = RobertaClass()
model.to(device)


input_ids = training_set_set[10]['ids'].unsqueeze(0).to(device)
input_mask = training_set_set[10]['mask'].unsqueeze(0).to(device)
token_type_ids = training_set_set[10]['token_type_ids'].unsqueeze(0).to(device)

output = model(input_ids, input_mask, token_type_ids)
output

loss_fn = nn.BCEWithLogitsLoss()
loss_fn = loss_fn.to(device)
loss = loss_fn(output, training_set_set[10]['targets'].unsqueeze(0).to(device))
loss

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.7762, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

## Training
### Hugginsfaces Trainer Class

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    logging_steps=10,
 ## ---
    report_to="tensorboard",
 )

def compute_metrics(p):
      pred, labels = p
      print(pred, labels,sep=" | ")
      pred = torch.round(pred)
      accuracy = ((pred == labels).sum(axis=1) == 5).sum() / len(labels)
      
      return {
         'accuracy': accuracy
      }

def compute_metrics_f1(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

class CustomTrainer(Trainer):
   def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
      labels = inputs.pop("labels")
      inputs["input_ids"] = inputs["input_ids"].squeeze(1)
      inputs["attention_mask"] = inputs["attention_mask"].squeeze(1)
      inputs["token_type_ids"] = inputs["token_type_ids"].squeeze(1)

      outputs = model(**inputs)
      loss = nn.BCEWithLogitsLoss()(outputs, labels)
      return (loss, outputs) if return_outputs else loss
   
trainer = CustomTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=validation_set,
   data_collator=data_collator,
   compute_metrics=compute_metrics, 
)

/Users/robertlange/Desktop/Uni/Semeval_Task/.venv/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer.train()

TypeError: forward() got an unexpected keyword argument 'labels'

### Custom Trainings Loop

In [ ]:
def calc_accuracy(X, y):
    y_pred = torch.sigmoid(X)
    y_pred = torch.round(y_pred)
    correct = (y_pred == y).float()
    accuracy = correct.sum() / len(correct)
    return accuracy

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

def train(epoch):
    model.train()
    for n,data in enumerate(train_dataloader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if n%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            accuracy = calc_accuracy(outputs, targets)
            print(f'Accuracy: {accuracy/5}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

In [ ]:
train(1)

RuntimeError: stack expects each tensor to be equal size, but got [31] at entry 0 and [14] at entry 1

## Validation
### With Huggingfaces Trainer Clasee

In [20]:
predictions = trainer.predict(tokenized_val)
pred = np.argmax(predictions.predictions, axis=-1)

TypeError: forward() got an unexpected keyword argument 'labels'

In [22]:
label_map = {
    'LABEL_0': 'Anger',
    'LABEL_1': 'Fear',
    'LABEL_2': 'Joy',
    'LABEL_3': 'Sadness',
    'LABEL_4': 'Surprise'
}

cm = confusion_matrix(predictions.label_ids, preds)

# label_map to labels
labels = [label_map[f'LABEL_{i}'] for i in range(len(label_map))]

# Confusion Matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix with Label Names')
plt.show()

NameError: name 'predictions' is not defined

### For Custom Trainings Loop

In [ ]:
# Validation
accuracy = 0
model.eval()
for n, data in enumerate(val_dataloader, 0):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.float)
    outputs = model(ids, mask, token_type_ids)
    loss = loss_fn(outputs, targets)
    accuracy += calc_accuracy(outputs, targets)
    if n%10==0:
        print(f'Validation Loss:  {loss.item()}')
n = len(val_dataloader)
print(f'Validation Accuracy: {(accuracy/n)/5}')

Validation Loss:  0.3331502377986908
Validation Loss:  0.5092226266860962
Validation Loss:  0.14787401258945465
Validation Loss:  0.3273431360721588
Validation Loss:  0.47030529379844666
Validation Loss:  0.1682194173336029
Validation Loss:  0.2163282334804535
Validation Loss:  0.14365515112876892
Validation Loss:  0.9091426730155945
Validation Loss:  0.20303049683570862
Validation Loss:  0.21051320433616638
Validation Loss:  0.9785973429679871
Validation Loss:  0.24564990401268005
Validation Loss:  0.18833395838737488
Validation Loss:  0.48059409856796265
Validation Loss:  0.24065573513507843
Validation Loss:  0.15683436393737793
Validation Loss:  0.5673535466194153
Validation Loss:  0.1696753203868866
Validation Loss:  0.1933712661266327
Validation Loss:  0.14872173964977264
Validation Loss:  0.1307135969400406
Validation Loss:  0.6664771437644958
Validation Loss:  0.5279530882835388
Validation Loss:  0.5955384373664856
Validation Loss:  0.16938245296478271
Validation Loss:  0.671253

In [ ]:
# test the model
input_ids = training_set[15]['ids'].unsqueeze(0).to(device)
input_mask = training_set[15]['mask'].unsqueeze(0).to(device)
token_type_ids = training_set[15]['token_type_ids'].unsqueeze(0).to(device)

output = model(input_ids, input_mask, token_type_ids)
out_ = torch.round(output.sigmoid())
print(training_set[15]['text'])
print(out_)
print(training_set[15]['targets'])


After scratching my head, I headed to the front yard, where there are no trees, on the off chance that something truly weird had caused a limb to plop down on my house.
tensor([[0., 1., 0., 0., 1.]], grad_fn=<RoundBackward0>)
tensor([0., 1., 0., 0., 1.])
